# Word processing

In [1]:
import json

In [2]:
filename = 'example.jsonl'

In [3]:
with open(filename, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 3:
            break
        if line.strip():
            obj = json.loads(line)
            print(json.dumps(obj, indent=2))
        

{
  "id": 1,
  "name": "Alice",
  "age": 30,
  "status": "active"
}
{
  "id": 2,
  "name": "Bob",
  "age": 25,
  "status": "inactive"
}
{
  "id": 3,
  "name": "Charlie",
  "age": 35,
  "status": "active"
}
